In [24]:
import pandas as pd
import numpy as np
from scipy.optimize import linear_sum_assignment

In [25]:
MOT = pd.read_csv("ADL-Rundle-6\det\Yolov5s\det.csv")
MOT.head()

,frame,id,bb_left,bb_top,bb_width,bb_height,conf,x,y,z
0,1,-1,1695,386,136,342,0.760947,-1,-1,-1
1,1,-1,255,459,103,246,0.676300,-1,-1,-1
2,1,-1,1885,380,34,202,0.628092,-1,-1,-1
3,1,-1,2,306,81,614,0.575901,-1,-1,-1
4,1,-1,1284,463,74,195,0.575504,-1,-1,-1


In [26]:
# Code CC CV depuis internet https://stackoverflow.com/questions/25349178/calculating-percentage-of-bounding-box-overlap-for-image-detector-evaluation
def get_iou(bb1, bb2):
    """
    Calculate the Intersection over Union (IoU) of two bounding boxes.

    Parameters
    ----------
    bb1 : dict
        Keys: {'x1', 'x2', 'y1', 'y2'}
        The (x1, y1) position is at the top left corner,
        the (x2, y2) position is at the bottom right corner
    bb2 : dict
        Keys: {'x1', 'x2', 'y1', 'y2'}
        The (x, y) position is at the top left corner,
        the (x2, y2) position is at the bottom right corner

    Returns
    -------
    float
        in [0, 1]
    """
    assert bb1['x1'] <= bb1['x2']
    assert bb1['y1'] <= bb1['y2']
    assert bb2['x1'] <= bb2['x2']
    assert bb2['y1'] <= bb2['y2']

    # determine the coordinates of the intersection rectangle
    x_left = max(bb1['x1'], bb2['x1'])
    y_top = max(bb1['y1'], bb2['y1'])
    x_right = min(bb1['x2'], bb2['x2'])
    y_bottom = min(bb1['y2'], bb2['y2'])

    if x_right < x_left or y_bottom < y_top:
        return 0.0

    # The intersection of two axis-aligned bounding boxes is always an
    # axis-aligned bounding box
    intersection_area = (x_right - x_left) * (y_bottom - y_top)

    # compute the area of both AABBs
    bb1_area = (bb1['x2'] - bb1['x1']) * (bb1['y2'] - bb1['y1'])
    bb2_area = (bb2['x2'] - bb2['x1']) * (bb2['y2'] - bb2['y1'])

    # compute the intersection over union by taking the intersection
    # area and dividing it by the sum of prediction + ground-truth
    # areas - the interesection area
    iou = intersection_area / float(bb1_area + bb2_area - intersection_area)
    assert iou >= 0.0
    assert iou <= 1.0
    return iou

In [27]:
# Donne les information bien ranger depuis un DF pour la matrice IoU
def GetDictFromDF(liste,pos):
    liste = liste.iloc[pos].to_list()
    return {"x1":liste[2] ,"y1":liste[3] , "x2":liste[2]  + liste[4] ,"y2":liste[3] + liste[5]}

In [28]:
# Rend un matrice carré en rajoutant une ligne ou un colonne
def make_square(matrix):
    rows, cols = matrix.shape
    if rows == cols:
        return matrix
    max_dim = max(rows, cols)
    square_matrix = np.zeros((max_dim, max_dim), dtype=matrix.dtype)
    square_matrix[:rows, :cols] = matrix
    return square_matrix

In [ ]:
CurrentFrame = 2
df = []

debug = True
nbFrames = len(MOT["frame"].unique())
posID = 0

IDList =  [1] * len(MOT[MOT["frame"] == CurrentFrame-1]) # Définit que les ID des objects de la première images sont 1,2,3,4,.....
for i in range(1,len(IDList)):
    IDList[i] += IDList[i-1]
    
IDMax = max(IDList)
IDList = [IDList] #Garde en memoire toutes les étapes

while(CurrentFrame != nbFrames):
    CurrentTrackedList = MOT[MOT["frame"] == CurrentFrame-1]
    NewTrackedList = MOT[MOT["frame"] == CurrentFrame]
    MatriceIoU = []
    
    for lig in range(len(NewTrackedList)):
        MatriceIoU.append([])
        value1forIoU = GetDictFromDF(NewTrackedList,lig)
        for col in range(len(CurrentTrackedList)):
            MatriceIoU[lig].append([])
            value2forIoU = GetDictFromDF(CurrentTrackedList,col)
            MatriceIoU[lig][col] = get_iou(value1forIoU,value2forIoU)
            
    MatriceIoU = np.array(MatriceIoU)
    Mshape = MatriceIoU.shape
    MatriceIoU = make_square(MatriceIoU)
    matcpy = MatriceIoU.copy()
    
    maximum = matcpy.max()
    matcpy = maximum - matcpy
    poxlist = linear_sum_assignment(matcpy)[1] # Récupération des meilleurs résultats

    if Mshape[0] > Mshape[1]:
        pl = []
        for i in range(len(poxlist)):
            if MatriceIoU[i][poxlist[i]] != 0:
                pl.append(poxlist[i])
            else:
                pl.append(-1)
        poxlist = pl
    newIDList = [0] * len(poxlist)
    
    
    for i in range(len(poxlist)):
        if poxlist[i] < len(IDList[posID]) and poxlist[i] != -1:
            newIDList[i] = IDList[posID][poxlist[i]]
        else: 
            newIDList[i] = IDMax
            IDMax += 1
            
    print("---------")
    print(newIDList)
    print(poxlist)
    posID += 1
    IDList.append(newIDList)
    CurrentFrame += 1

---------
[2, 1, 3, 4, 5, 5, 6, 7]
[1 0 2 3 6 4 5 7]
---------
[2, 1, 3, 4, 5, 5, 6, 8, 7]
[0 1 2 3 5 4 6 8 7]
---------
[2, 3, 4, 1, 5, 8, 6, 5, 7]
[0 2 3 1 4 7 6 5 8]
---------
[2, 3, 5, 1, 4, 9, 8, 10, 6]
[0, 1, 4, 3, 2, -1, 5, -1, 6]
---------
[3, 2, 4, 1, 5, 9, 8]
[1 0 4 3 2 5 6]
---------
[3, 2, 4, 1, 5, 9, 11]
[0, 1, 2, 3, 4, 5, -1]
---------
[3, 2, 4, 1, 9, 5]
[0 1 2 3 5 4]
---------
[3, 4, 2, 1, 9, 5]
[0 2 1 3 4 5]
---------
[3, 4, 1, 2, 9, 12]
[0, 1, 3, 2, 4, -1]
---------
[3, 4, 2, 1, 9, 12]
[0 1 3 2 4 5]
---------
[3, 4, 2, 1, 9, 13]
[0, 1, 2, 3, 4, -1]
---------
[3, 4, 13, 2, 9, 1]
[0 1 5 2 4 3]
---------
[3, 4, 13, 2, 9, 14]
[0, 1, 2, 3, 4, -1]
---------
[3, 4, 2, 14, 13, 9]
[0 1 3 5 2 4]
---------
[3, 4, 2, 14, 13, 15]
[0, 1, 2, 3, 4, -1]
---------
[3, 4, 14, 2, 13]
[0 1 3 2 4]
---------
[3, 4, 14, 2, 13]
[0 1 2 3 4]
---------
[3, 4, 14, 2, 13, 16]
[0 1 2 3 4 5]
---------
[3, 14, 4, 2, 16, 13, 17, 18]
[0 2 1 3 5 4 6 7]
---------
[3, 14, 4, 16, 2, 19, 13, 20]
[0, 1, 2, 4,

In [30]:
import cv2
import os

frames_folder = "ADL-Rundle-6/img1/"

# Lire toutes les frames du dossier
frames = sorted([os.path.join(frames_folder, f) for f in os.listdir(frames_folder) if f.endswith('.jpg')])

# Lire et afficher chaque frame avec des rectangles
framepos = 1
for frame_path in frames:
    # Charger l'image
    frame = cv2.imread(frame_path)
    height, width = frame.shape[:2]
    
    rectangles = []
    for i in range(len(MOT[MOT["frame"] == framepos])):
        rectangles.append(GetDictFromDF(MOT[MOT["frame"] == framepos],i))
    
    assert(len(rectangles) == len(MOT[MOT["frame"] == framepos]))
    # Ajouter des rectangles
    posrec = 0
    for rect in rectangles:
        x1, y1, x2, y2 = int(rect["x1"]), int(rect["y1"]), int(rect["x2"]), int(rect["y2"]) # Regarder par rapport au axes
        color = (0,255,0)
        thickness = 2
        cv2.rectangle(frame, (x1, y1), (x2, y2), color, thickness)
        font = cv2.FONT_HERSHEY_SIMPLEX
        font_scale = 3
        font_thickness = 3
        text_color = (255, 0, 0)
        if framepos >= len(IDList) -1:
            break
        print(framepos)
        print(posrec)
        print(len(rectangles))
        print(IDList[framepos-1])
        print("---------")
        title = str(IDList[framepos-1][posrec])
        cv2.putText(frame, title, (x1, y1), font, font_scale, text_color, font_thickness, cv2.LINE_AA)
        cv2.putText(frame, str(framepos), (30, 30), font, 1, (255, 255, 255), 1, cv2.LINE_AA)
        cv2.putText(frame, str(len(MOT[MOT["frame"] == framepos])), (30, 70), font, 1, (255, 255, 255), 1, cv2.LINE_AA)
        posrec += 1
    
    framepos +=1
    # Afficher la frame
    cv2.imshow('Video Frame', frame)
    
    TimeToweight = 40
    # Attendre 30 ms pour simuler un FPS de ~30
    if cv2.waitKey(TimeToweight) & 0xFF == ord('q'):  # Quitter avec 'q'
        break

cv2.destroyAllWindows()


1
0
5
[1, 2, 3, 4, 5]
---------
1
1
5
[1, 2, 3, 4, 5]
---------
1
2
5
[1, 2, 3, 4, 5]
---------
1
3
5
[1, 2, 3, 4, 5]
---------
1
4
5
[1, 2, 3, 4, 5]
---------
2
0
8
[2, 1, 3, 4, 5, 5, 6, 7]
---------
2
1
8
[2, 1, 3, 4, 5, 5, 6, 7]
---------
2
2
8
[2, 1, 3, 4, 5, 5, 6, 7]
---------
2
3
8
[2, 1, 3, 4, 5, 5, 6, 7]
---------
2
4
8
[2, 1, 3, 4, 5, 5, 6, 7]
---------
2
5
8
[2, 1, 3, 4, 5, 5, 6, 7]
---------
2
6
8
[2, 1, 3, 4, 5, 5, 6, 7]
---------
2
7
8
[2, 1, 3, 4, 5, 5, 6, 7]
---------
3
0
9
[2, 1, 3, 4, 5, 5, 6, 8, 7]
---------
3
1
9
[2, 1, 3, 4, 5, 5, 6, 8, 7]
---------
3
2
9
[2, 1, 3, 4, 5, 5, 6, 8, 7]
---------
3
3
9
[2, 1, 3, 4, 5, 5, 6, 8, 7]
---------
3
4
9
[2, 1, 3, 4, 5, 5, 6, 8, 7]
---------
3
5
9
[2, 1, 3, 4, 5, 5, 6, 8, 7]
---------
3
6
9
[2, 1, 3, 4, 5, 5, 6, 8, 7]
---------
3
7
9
[2, 1, 3, 4, 5, 5, 6, 8, 7]
---------
3
8
9
[2, 1, 3, 4, 5, 5, 6, 8, 7]
---------
4
0
9
[2, 3, 4, 1, 5, 8, 6, 5, 7]
---------
4
1
9
[2, 3, 4, 1, 5, 8, 6, 5, 7]
---------
4
2
9
[2, 3, 4, 1, 5, 8, 6, 